REMEMBER: FIRST CREATE A COPY OF THIS FILE WITH A UNIQUE NAME AND DO YOUR WORK THERE. AND MAKE SURE YOU COMMIT YOUR CHANGES TO THE `hw3_submissions` BRANCH.

# Assignment 3 | Cleaning and Exploring Data with Pandas



<img src="data/scoreCard.jpg" width=250>

In this assignment, you will investigate restaurant food safety scores for restaurants in San Francisco. Above is a sample score card for a restaurant. The scores and violation information have been made available by the San Francisco Department of Public Health. 

## Loading Food Safety Data


There are 2 files in the data directory:
1. business.csv containing food establishments in San Francisco
1. inspections.csv containing retaurant inspections records

Let's start by loading them into Pandas dataframes.  One of the files, business.csv, has encoding (ISO-8859-1), so you will need to account for that when reading it.

### Question 1

#### Question 1a
Read the two files noted above into two pandas dataframes named `bus` and `ins`, respectively. Print the first 5 rows of each to inspect them.


In [2]:
import pandas as pd

In [3]:
bus = pd.read_csv('data/businesses.csv', encoding='ISO-8859-1')
ins = pd.read_csv('data/inspections.csv')

In [4]:
bus.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494
3,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN
4,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051


In [5]:
ins.head(5)

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine


## Examining the Business data

From its name alone, we expect the `businesses.csv` file to contain information about the restaurants. Let's investigate this dataset.

### Question 2

#### Question 2a: How many records are there?

In [106]:
print("Number of records: " + str(len(bus.index)))

Number of records: 6315


#### Question 2b: How many unique business IDs are there?  

In [107]:
print("Number of unique business IDs: " + str(len(bus['business_id'].unique())))

Number of unique business IDs: 6315


#### Question 2c: What are the 5 most common businesses by name, and how many are there in San Francisco?

In [8]:
# value_counts sort the counts by default
print(bus.value_counts(subset=['name'])[:5])

name                      
STARBUCKS COFFEE              72
PEET'S COFFEE & TEA           24
MCDONALDS                     12
WALGREENS                     11
SAN FRANCISCO SOUP COMPANY    11
dtype: int64


## Zip code

Next, let's  explore some of the variables in the business table. We begin by examining the postal code.

### Question 3

#### Question 3a
How are the zip code values stored in python (i.e. data type)?

To answer this you might want to examine a particular entry.

In [9]:
type(bus.loc[0,'postal_code'])

str

The zipcodes are stored as strings

#### Question 3b

What are the unique values of postal_code?

In [10]:
print(bus['postal_code'].unique())

['94104' '94109' '94133' '94110' '94122' '94103' '94115' '94131' '94111'
 '94117' '94107' '94108' '94102' '94132' '94105' '94134' '94124' '94116'
 '94121' '94118' '94112' '94123' '94114' '94127' '941102019' '941' '94545'
 '94066' '941033148' nan '94158' '95105' '94013' '94130' 'CA' '92672'
 '94120' '94143' '94609' '94101' '00000' '94188' '94621' '94014' '94129'
 '94602' 'Ca']


#### Question 3c

Let's say we decide to exclude the businesses that have no zipcode for our analysis (which might include food trucks for example).  Use the list of valid 5-digit zip codes below to create a new dataframe called bus_valid, with only businesses whose postal_codes show up in this list of valid zipcodes. How many businesses are there in this new dataframe?

In [108]:
valid_zip = ["94102", "94103", "94104", "94105", "94107", "94108",
            "94109", "94110", "94111", "94112", "94114", "94115",
            "94116", "94117", "94118", "94121", "94122", "94123", 
            "94124", "94127", "94131", "94132", "94133", "94134"]

In [110]:
bus_valid = bus[bus['postal_code'].isin(valid_zip)]
print('Number of businesses with valid zipcode: ' + str(len(bus_valid.index)))

Number of businesses with valid zipcode: 5999


## Latitude and Longitude

Another aspect of the data we want to consider is the prevalence of missing values. If many records have missing values then we might be concerned about whether the nonmissing values are representative of the population.

### Question 4
 
Consider the longitude and latitude in the business DataFrame. 

#### Question 4a

How many businesses are missing longitude values, working with only the businesses that are in the list of valid zipcodes?

In [111]:
print('Number of businesses missing longitudes: ' + str(len(bus_valid[pd.isnull(bus_valid['longitude'])])))

Number of businesses missing longitudes: 2483


#### Question 4b

Create a new dataframe with one row for each valid zipcode. The dataframe should include the following three columns:

1. `postal_code`: Contains the zip codes in the `validZip` variable above.
2. `null_lon`: The number of businesses in that zipcode with missing `longitude` values.
3. `not_null_lon`: The number of businesses without missing `longitude` values.

In [112]:
# count method only counts not_null values
not_null_df = bus_valid[['postal_code', 'longitude']].groupby('postal_code').count()
# count the null values by counting the business ids
null_df = bus_valid[pd.isnull(bus_valid['longitude'])][['postal_code', 'business_id']].groupby('postal_code').count()
# outer join on index
count_df = pd.merge(not_null_df, null_df, how="outer", left_index=True, right_index=True)
# rename the columns
count_df = count_df.rename(columns={"longitude": "not_null_lon", "business_id": "null_lon"})
count_df

,not_null_lon,null_lon
postal_code,,
94102,262,196
94103,306,252
94104,74,59
94105,126,100
94107,204,247
94108,142,82
94109,237,146
94110,334,251
94111,167,115


#### 4c. Do any zip codes appear to have more than their 'fair share' of missing longitude? 

To answer this, you will want to compute the proportion of missing longitude values for each zip code, and print the proportion missing longitude, and print the top five zipcodes in descending order of proportion missing postal_code.


In [113]:
# compute and print proportion
count_df['prop_null_lon'] = count_df['null_lon'] / (count_df['null_lon'] + count_df['not_null_lon'])
count_df

,not_null_lon,null_lon,prop_null_lon
postal_code,,,
94102,262,196,0.427948
94103,306,252,0.451613
94104,74,59,0.443609
94105,126,100,0.442478
94107,204,247,0.547672
94108,142,82,0.366071
94109,237,146,0.381201
94110,334,251,0.429060
94111,167,115,0.407801


In [114]:
# find the five zipcodes with largest proportion of missing longitude
zipcode_list = count_df.sort_values('prop_null_lon')[:5].index.tolist()
print('The top five zipcodes in proportion of missing longitude (descending order):')
for i in zipcode_list:
    print(i)

The top five zipcodes in proportion of missing longitude (descending order):
94131
94112
94123
94115
94133


# Investigate the inspection data

Let's now turn to the inspection DataFrame. Earlier, we found that `ins` has 4 columns, these are named `business_id`, `score`, `date` and `type`.  In this section, we determine the granularity of `ins` and investigate the kinds of information provided for the inspections. 

### Question 5

#### Question 5a
As with the business data, assess whether there is one inspection record for each business, by counting how many rows are in the data and how many unique businesses there are in the data. If they are exactly the same number, it means there is only one inspection per business, clearly.

In [61]:
# count rows
print('Number of rows: ' + str(len(ins.index)))
# count unique business ids
print('Number of unique businesses: ' + str(len(ins['business_id'].unique())))

Number of rows: 15430
Number of unique businesses: 5730


There are more inspection records than unique businesses.

#### Question 5b

What values does `type` take on? How many occurrences of each value is in the DataFrame? Create a new dataframe named `ins2` by copying `ins` and keeping only records with values of `type` that occur more than 10 times in the original table. In other words, eliminate records that have values of `type` that occur rarely (< 10 times). Check the result to make sure rare types are eliminated.

In [77]:
# count the number of each value of type
type_count = ins[['type', 'business_id']].groupby('type').count()
type_count

,business_id
type,
complaint,1
routine,15429


In [90]:
# create a list with type values occuring >= 10
type_list = type_count[type_count['business_id']>=10].index.tolist()
# filter ins with the value list
ins2 = ins[ins.type.isin(type_list)]
ins2

,business_id,score,date,type,year
0,10,82,20160503,routine,2016
1,10,94,20140729,routine,2014
2,10,92,20140114,routine,2014
3,19,94,20160513,routine,2016
4,19,94,20141110,routine,2014
...,...,...,...,...,...
15425,89515,98,20161103,routine,2016
15426,89701,100,20161206,routine,2016
15427,89790,96,20161129,routine,2016
15428,89803,100,20161215,routine,2016


#### Question 5c

Since the data was stored in a .csv file, the dates are formatted as strings such as `20160503`. Once we read in the data, we would like to have dates in an appropriate format for analysis. Add a new column called `year` by capturing the first four characters of the date column. 

Hint: we have seen multiple ways of doing this in class, includings `str` operations, `lambda` functions, `datetime` operations, and others. Choose the method that works best for you :)

In [86]:
type(ins.loc[0, 'date'])

numpy.int64

In [87]:
# because dates are stored as integers and share the same format
# we can get the year using integer division
ins['year'] = ins['date']//10000
ins

,business_id,score,date,type,year
0,10,82,20160503,routine,2016
1,10,94,20140729,routine,2014
2,10,92,20140114,routine,2014
3,19,94,20160513,routine,2016
4,19,94,20141110,routine,2014
...,...,...,...,...,...
15425,89515,98,20161103,routine,2016
15426,89701,100,20161206,routine,2016
15427,89790,96,20161129,routine,2016
15428,89803,100,20161215,routine,2016


#### Question 5d

What range of years is covered in this data set? Are there roughly same number of inspections each year? Try dropping records for any years with less than 50 inspections and store the result in a new dataframe named `ins3`.

In [89]:
# count number of inspections each year
year_count = ins.groupby('year').count()
year_count

,business_id,score,date,type
year,,,,
2013,38,38,38,38
2014,5629,5629,5629,5629
2015,3923,3923,3923,3923
2016,5840,5840,5840,5840


As shown above, the years covered is 2013 to 2016. The number of inspections varies across the years.

In [93]:
# get a list of years with more than 50 inspections
year_list = year_count[year_count['business_id']>=50].index.tolist()
# filter ins with the value list
ins3 = ins[ins.year.isin(year_list)]
ins3

,business_id,score,date,type,year
0,10,82,20160503,routine,2016
1,10,94,20140729,routine,2014
2,10,92,20140114,routine,2014
3,19,94,20160513,routine,2016
4,19,94,20141110,routine,2014
...,...,...,...,...,...
15425,89515,98,20161103,routine,2016
15426,89701,100,20161206,routine,2016
15427,89790,96,20161129,routine,2016
15428,89803,100,20161215,routine,2016


Let's examine only the inspections for one year: 2016. This puts businesses on a more equal footing because [inspection guidelines](https://www.sfdph.org/dph/eh/Food/Inspections.asp) generally refer to how many inspections should occur in a given year.

In [97]:
ins_2016 = ins[ins.year == 2016]
ins_2016

,business_id,score,date,type,year
0,10,82,20160503,routine,2016
3,19,94,20160513,routine,2016
6,24,98,20161005,routine,2016
7,24,96,20160311,routine,2016
10,31,84,20160816,routine,2016
...,...,...,...,...,...
15425,89515,98,20161103,routine,2016
15426,89701,100,20161206,routine,2016
15427,89790,96,20161129,routine,2016
15428,89803,100,20161215,routine,2016


### Question 6

#### Question 6a

Merge the business and 2016 inspections data, keeping all businesses regardless of whether they show up in the inspections file. Show the first several rows of the resulting dataframe.

In [100]:
merged = pd.merge(bus, ins_2016, how="outer", on='business_id')
merged.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number,score,date,type,year
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044,82.0,20160503.0,routine,2016.0
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262,94.0,20160513.0,routine,2016.0
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,98.0,20161005.0,routine,2016.0
3,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,96.0,20160311.0,routine,2016.0
4,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN,84.0,20160816.0,routine,2016.0


#### Question 6b
Print the 20 lowest rated businesses names, their addresses, and their ratings.

In [105]:
# the sort_value method will put NaN values in the end by default
names = merged.sort_values('score', ascending=True).head(20)['name'].tolist()
# print out the names
print("The 20 lowest rated business in 2016:")
for n in names:
    print(n)

The 20 lowest rated business in 2016:
GOLDEN RIVER RESTAURANT
POKI TIME
CRAZY PEPPER
UNCLE CAFE
HING WANG BAKERY
GOLDEN WOK
TOP'S CAFE
L & G VIETNAMESE SANDWICH
RED A BAKERY
SAN TUNG RESTAURANT LLC
MA'S DIM SUM & CAFE
NEW GARDEN RESTAURANT, INC.
KOBANI MEDITERANEAN GRILL
JOANIE'S DINER INC.
PAK NAM THAI CUISINE
JADE GARDEN
ITHAI
VALLARTA'S TACO BAR
HONG KONG LOUNGE
WOLFES LUNCH


## Done!

Now commit this notebook to your `hw3_submissions` branch, push it to your GitHub repo, and open a PR!